In [1]:
# Compute the 3D trvavel time volumes for a 1D velocity model

In [2]:
## Import modules

from pyrocko import cake
import numpy as np
import pandas as pd
from os.path import join
from time import time


In [3]:
## Input paramters

subarray = "A"
phase = "P"

modpath = "/Volumes/OmanData/geophones_no_prefilt/data/vp_1d_A.nd"
stapath = "/Volumes/OmanData/geophones_no_prefilt/data/stations.csv"
gridpath = "/Volumes/OmanData/geophones_no_prefilt/data/traveltimes_subarrays/grid_params_A.csv"

dirname_out = "/Volumes/OmanData/geophones_no_prefilt/data/traveltimes_subarrays"

In [4]:
## Read the grid parameters
griddf = pd.read_csv(gridpath, sep=" ")

eastmin = griddf["min_easting"][0]
eastmax = griddf["max_easting"][0]
eastinc = griddf["easting_inc"][0]
northmin = griddf["min_northing"][0]
northmax = griddf["max_northing"][0]
northinc = griddf["northing_inc"][0]
depthmin = griddf["min_depth"][0]
depthmax = griddf["max_depth"][0]
depthinc = griddf["depth_inc"][0]


In [5]:
## Load the station coordinates
stadf = pd.read_csv(stapath, sep=" ")

if subarray == "A":
    stadf = stadf.loc[stadf["subarray"] == "A"]
elif subarray == "B":
    stadf = stadf.loc[stadf["subarray"] == "B"]

In [6]:
# Load the velocity model
model = cake.load_model(modpath)

In [7]:
# Define the spatial grid
numea = int((eastmax - eastmin) / eastinc+1)
numno = int((northmax - northmin) / eastinc+1)
numde = int((depthmax - 0.0) / depthinc+1)

eagrid = np.linspace(eastmin, eastmax, numea)
nogrid = np.linspace(northmin, northmax, numno)
degrid = np.linspace(0.0, depthmax, numde)

In [8]:
# Define the phase to use.
phases =[cake.PhaseDef('P'), cake.PhaseDef('p')]

In [9]:
# Compute the travel times for each station
print("Computing travel times...")
numst = len(stadf)
print(f"Number of stations to compute : {numst}")

for stind, row in stadf.iterrows():
    stname = row["name"]
    stea = row["easting"]
    stno = row["northing"]

    print(stname)

    # Determine if the station is in the grid
    if stea < eastmin or stea > eastmax or stno < northmin or stno > northmax:
        print(f"Station {stname} is outside the grid. Skipping...")
        continue
    
    begin  = time()
    ttimes = np.zeros((numde, numno, numea))

    # Compute the travel times for each grid point
    for deind, de in enumerate(degrid):
        for noind, no in enumerate(nogrid):
            for eaind, ea in enumerate(eagrid):
                dist = np.sqrt((stea-ea)**2 + (stno-no)**2)
                arrivals = model.arrivals([dist*cake.m2d], phases=phases, zstart=de)
                ttime = arrivals[0].t
                ttimes[deind, noind, eaind] = ttime

    # Save the travel times
    print("Saving travel times...")

    if phase == "P":
        outpath = join(dirname_out, f"ptimes_{stname}.npy")
    elif phase == "S":
        outpath = join(dirname_out, f"stimes_{stname}.npy")
    else:
        raise ValueError("phase must be P or S!")

    ## Save the data with metadata
    np.save(outpath, ttimes)
    print(f"Saved to {outpath}.")

    end = time()
    print(f"Time elapsed: {end-begin} seconds.")

Computing travel times...
Number of stations to compute : 19
A01
Saving travel times...
Saved to /Volumes/OmanData/geophones_no_prefilt/data/traveltimes_subarrays/ptimes_A01.npy.
Time elapsed: 374.21741127967834 seconds.
A02
Saving travel times...
Saved to /Volumes/OmanData/geophones_no_prefilt/data/traveltimes_subarrays/ptimes_A02.npy.
Time elapsed: 383.8898491859436 seconds.
A03
Saving travel times...
Saved to /Volumes/OmanData/geophones_no_prefilt/data/traveltimes_subarrays/ptimes_A03.npy.
Time elapsed: 378.29045820236206 seconds.
A04
Saving travel times...
Saved to /Volumes/OmanData/geophones_no_prefilt/data/traveltimes_subarrays/ptimes_A04.npy.
Time elapsed: 372.8554768562317 seconds.
A05
Saving travel times...
Saved to /Volumes/OmanData/geophones_no_prefilt/data/traveltimes_subarrays/ptimes_A05.npy.
Time elapsed: 368.3323850631714 seconds.
A06
Saving travel times...
Saved to /Volumes/OmanData/geophones_no_prefilt/data/traveltimes_subarrays/ptimes_A06.npy.
Time elapsed: 367.840883

In [10]:
# # Plot the travel times
# from matplotlib import pyplot as plt    

# fig, axes = plt.subplots(nrows=1, ncols=2)
# axes[0].imshow(ttimes[0,:,:], origin="lower", extent=[eastmin, eastmax, northmin, northmax], aspect="equal")

# noind_st = np.argmin(np.abs(nogrid - stadf["northing"][0]))
# axes[1].imshow(ttimes[:,noind_st,:], origin="upper", extent=[eastmin, eastmax, depthmax, 0.0], aspect="equal")
